# Demo of our DuReader machine reading comprehension

### Configuration
Since we can't wrap the arguments with argparse on the notebook, we put all the parameters in the config.py file (like this).

In [1]:
# '''Path parameter'''
# path = os.getcwd()
# vocab_dir = path + '/data/vocab/'
# train_files = [ './data/demo//search.train.json']
# dev_files = ['./data/demo//search.dev.json']
# test_files = ['./data/demo//search.test.json']
# pretrained_word_path = None
# pretrained_char_path = None
# model_dir = path + '/data/models/'
# 
# # --train --batch_size 16 --learning_rate 1e-3 --optim adam --decay 0.9999 --weight_decay 1e-5
# #  --max_norm_grad 5.0 --dropout 0.0 --head_size 1 --hidden_size 64 --epochs 20 --gpu 0
# '''Train settings'''
# algo = 'qanet'
# epochs = 20
# loss_type = 'cross_entropy'
# batch_size = 16
# optim = 'adam'
# fix_pretrained_vector = True
# 
# learning_rate = 1e-3
# dropout = 0.5
# weight_decay = 1e-5
# l2_norm = 3e-7
# clip_weight = True
# max_norm_grad = 5.0
# decay = 0.9999
# gpu  = '0'
# 
# 
# '''Model setting'''
# max_p_num = 5
# max_p_len = 400
# max_q_len = 60
# max_ch_len =20
# max_a_len = 200
# word_embed_size = 150
# char_embed_size = 32
# head_size = 1
# hidden_size = 64
# use_position_attn = False

### Dataloader & Vocab

* We use the already written dataloader and vocab class to encapsulate the data into the form we want and build Vocab from the characters and words in the dataset.
* There are these attributes in class of dataloader: 'question_type', 'segmented_question', 'question_id', 'question_tokens', 'answer_spans', 'match_scores', 'fact_or_opinion', 'passages', 'segmented_answers', 'fake_answers', 'question_chars', 'answer_passages', 'answer_docs', 'documents', 'answers', 'question'
* There are these attributes in class of Vocab: id2word, id2char, lower, word2id, char2id, char_cnt, word_cnt

In [2]:
import config
import os
import pickle
import config
from dataloader.OurDataLoader import DataLoader
from VocabBuild.OurVocab import Vocab
from model.OurModel import Model

for data_path in [config.train_files[0], config.dev_files[0], config.test_files[0]]:
    assert os.path.exists(data_path), '{} file does not exist.'.format(data_path)

print('Preparing the directories...')
for dir_path in [config.train_files[0], config.dev_files[0], config.test_files[0]]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

print('Building vocabulary...')
'''
'question_type', 'segmented_question', 'question_id', 'question_tokens', 'answer_spans', 'match_scores', 
'fact_or_opinion', 'passages', 'segmented_answers', 'fake_answers', 'question_chars', 'answer_passages', 
'answer_docs', 'documents', 'answers', 'question'
'''
dataloader = DataLoader(config.max_p_num, config.max_p_len, config.max_q_len, config.max_ch_len,
                        config.train_files, config.dev_files, config.test_files) 
vocab = Vocab(lower=True) 
'''
id2word, id2char, lower, word2id, char2id, char_cnt, word_cnt
'''
for word in dataloader.word_iter('train'):
    vocab.add_word(word)  
    [vocab.add_char(ch) for ch in word] #

# Filtering the words in the dictionary from the words
unfiltered_vocab_size = vocab.word_size()
vocab.filter_words_by_cnt(min_cnt=2) 
filtered_num = unfiltered_vocab_size - vocab.word_size()

print('After filter {} tokens, the final vocab size is {}, char size is {}'.
      format(filtered_num,vocab.word_size(),vocab.char_size()))

unfiltered_vocab_char_size = vocab.char_size()
vocab.filter_chars_by_cnt(min_cnt=2)
filtered_char_num = unfiltered_vocab_char_size - vocab.char_size()
print('After filter {} chars from {} original chars, the final char vocab size is {}'
      .format(filtered_char_num,unfiltered_vocab_char_size,vocab.char_size()))

print('Assigning embeddings...')
if config.pretrained_word_path is not None:
    vocab.load_pretrained_word_embeddings(config.pretrained_word_path)
else:
    vocab.randomly_init_word_embeddings(config.word_embed_size)

if config.pretrained_char_path is not None:
    vocab.load_pretrained_char_embeddings(config.pretrained_char_path)
else:
    vocab.randomly_init_char_embeddings(config.char_embed_size)

print('Saving vocab...')
with open(os.path.join(config.vocab_dir, 'vocab.data'), 'wb') as fout:
    pickle.dump(vocab, fout)

/home/yxu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Preparing the directories...
Building vocabulary...
After filter 5225 tokens, the final vocab size is 5006, char size is 2903
After filter 669 chars from 2903 original chars, the final char vocab size is 2234
Assigning embeddings...
Saving vocab...


Next, we can look at what information is in the dataset.

In [3]:
sample = dataloader.train_set[0] # Choosing the 1st sample in class of dataloader

We can take a look at the questions contained in a sample and the type of problem.

In [4]:
print('rquestion_type :',sample['question_type'])
print('\rquestion:',''.join(sample['segmented_question']))
print('\rquestion_id:',sample['question_id'])
print('\ranswer_spans:',sample['answer_spans'][0])
print('\rmatch_scores:', sample['match_scores'][0])

rquestion_type : DESCRIPTION
question: 微信分享链接打开app
question_id: 91159
answer_spans: [0, 110]
match_scores: 0.8815165876777251


We found that there are four answers in this sample, two of which were adopted and two were not adopted.

In [5]:
print('\r','\rpassages 1:', ''.join([x for i in sample['passages'][0]['passage_chars'] for x in i]))
print('\rselect of passages 1:', sample['passages'][0]['is_selected'])
print('\r','\rpassages 2:', ''.join([x for i in sample['passages'][1]['passage_chars'] for x in i]))
print('\rselect of passages 2:', sample['passages'][1]['is_selected'])
print('\r','\rpassages 3:', ''.join([x for i in sample['passages'][2]['passage_chars'] for x in i]))
print('\rselect of passages 3:', sample['passages'][2]['is_selected'])
print('\r','\rpassages 4:', ''.join([x for i in sample['passages'][3]['passage_chars'] for x in i]))
print('\rselect of passages 3:', sample['passages'][2]['is_selected'])
print('\r','\rpassages 5:', ''.join([x for i in sample['passages'][4]['passage_chars'] for x in i]))
print('\rselect of passages 3:', sample['passages'][2]['is_selected'])

passages 1: iOS里,把一个页面链接分享给微信好友(会话),好友在微信里打开这个链接,也就是打开了一个网页,点击网页里的某个地方后(比如网页中“打开xx应用程序”的按钮),代码里怎么设置可以跳回到第三方app?知乎的ios客户端就有这种功能,在微信里分享链接后,点开链接,再点网页中的某处,就可以打开知乎客户端显示全部
select of passages 1: True
passages 2: 1首先打开腾讯开放平台,注册开放平台的一个账号,如下图所示。步骤阅读2登录后在”管理中心“中创建应用,如下图所示点击红线框选按钮。步骤阅读3选择创建应用的类型如下图所示,然后按钮提示填写应用的相关信息,提交进入审核。步骤阅读步骤阅读4应用审核通过后会在管理中心中显示”已上线“,然后点击应用的图标。步骤阅读5在应用详情中找到运营服务中的”微下载“,点击进入详情页面。步骤阅读6在详情页面中就可以看到系统生成的推广链接地址了,复制此地址就可以微信中转发了。步骤阅读END
select of passages 2: True
passages 3: app中分享到微信已经成功,但是通过分享的连接无法启动app。
select of passages 3: False
passages 4: 项目中有个需求,让用户可以从分享到微信的网页中启动自己的APP,如果本机没有安装该应用程序则打开应用商店并打开该程序在商店中的搜索结果页面。在这里跟大家分享一下:
select of passages 3: False
passages 5: 1将手机微信打开。步骤阅读2打开微信中的链接。步骤阅读3如图我们打开百度经验的链接,用微信浏览器进入网页以后,点击右上方如图符号。步骤阅读4进入选择页面,点击“在浏览器”打开。步骤阅读5如图会出现手机已安装的本地APP浏览器,我们选择一个自己想用的本地浏览器点击下方“仅一次”或“总是”都可以打开打开本地APP浏览器。步骤阅读6此时,我们就已经在微信浏览器中将本地APP浏览器打开了步骤阅读END
select of passages 3: False


Fortunately, Baidu has helped us screen out the correct answer and the wrong answer to this question.

In [6]:
print('\rfake_answers:', sample['fake_answers'][0])
print('-----------------------------------------------------------------------------------------------------------------------------------------')
print('\rreal answer:', sample['answers'][0])

fake_answers: 1将手机微信打开。步骤阅读2打开微信中的链接。步骤阅读3如图我们打开百度经验的链接,用微信浏览器进入网页以后,点击右上方如图符号。步骤阅读4进入选择页面,点击“在浏览器”打开。步骤阅读5如图会出现手机已安装的本地APP浏览器,我们选择一个自己想用的本地浏览器点击下方“仅一次”或“总是”都可以打开打开本地APP浏览器。步骤阅读6此时,我们就已经在微信浏览器中将本地APP浏览器打开了
-----------------------------------------------------------------------------------------------------------------------------------------
real answer: iOS里，把一个页面链接分享给微信好友（会话），好友在微信里打开这个链接,也就是打开了一个网页，点击网页里的某个地方后（比如网页中“打开xx应用程序”的按钮），代码里怎么设置可以跳回到第三方app？知乎的ios客户端就有这种功能，在微信里分享链接后，点开链接，再点网页中的某处，就可以打开知乎客户端 。


These five documents are the corpus corresponding to the problem. Our model is to learn how to search for the correct answer above in this five paragraphs.

In [7]:
print('Document1',''.join(sample['documents'][0]['paragraphs']))
print('Document2',''.join(sample['documents'][1]['paragraphs']))
print('Document3',''.join(sample['documents'][2]['paragraphs']))
print('Document4',''.join(sample['documents'][3]['paragraphs']))
print('Document5',''.join(sample['documents'][4]['paragraphs']))

Document1 iOS里,把一个页面链接分享给微信好友(会话),好友在微信里打开这个链接,也就是打开了一个网页,点击网页里的某个地方后(比如网页中“打开xx应用程序”的按钮),代码里怎么设置可以跳回到第三方app?知乎的ios客户端就有这种功能,在微信里分享链接后,点开链接,再点网页中的某处,就可以打开知乎客户端显示全部微信中不能用自定义url的方式,微信提供了打开第三方应用的接口:launch3rdApp谢。一般用自带浏览器可以调用起app没问题。微信里面能调出app的,是和腾讯有合作的应用,其他会被过滤掉。有一个公司的产品,叫 魔窗,免费可以接入的分享出去的是一个网页,前端人员与app客户端人员约定好一个头部协议。 当用户在网页的点击某个按钮,调用注册的协议头,就可以呼起安装的app。比如知乎app可能注册了 zhifu://xxx zhifu:// 就是约定好的xxx 是指参数 可以任意带这样访问后就可以呼起app了。目前已经解决直接从微信打开的链接可以直接点击下载APP的跳转,无需再通过另外浏览中打开才能点击下载,在微信中即可直接点击下载,需要的联系QQ77408026你可以去看看openinstall,它可以实现让用户分享链接让别的用户可以打开下载或者跳转APP。而且打开或者下载后还能获取到自己想传过去的参数。方法一:微信API--WXAppExtendObject.(参考app-微博)方法二:iOS 9 Universal Link.(参考app-蘑菇街)目前要先用浏览器打开,然后才能跳。有解决办法么下载知乎客户端 与世界分享知识、经验和见解相关Live 推荐
Document2 百度经验:jingyan.baidu.com我们都知道手游、APP想在微信分享时,下载的链接会被微信屏蔽,不能直接点击,那怎样才能得到微信认可的下载页面呢,这里就需要用到腾讯的开放平台,而你的应用也需要提交到这个平台。在这里可以得到微信认可的链接地址。接下来介绍相关方法。百度经验:jingyan.baidu.com百度经验:jingyan.baidu.com1 首先打开腾讯开放平台,注册开放平台的一个账号,如下图所示。 步骤阅读 2 登录后在”管理中心“中创建应用,如下图所示点击红线框选按钮。 步骤阅读 3 选择创建应用的类型如下图所示,然后按钮提示填写应用的相关信息,提交

### Model training & Evaluation

Now, we can train our QANet model.

In [8]:
import tensorflow as tf
import os
import pickle
import config
from dataloader.OurDataLoader import DataLoader
from VocabBuild.OurVocab import Vocab
from model.OurModel import Model


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu


print('Load vocab...')
with open(os.path.join(config.vocab_dir, 'vocab.data'), 'rb') as fin:
    vocab = pickle.load(fin)

print('Load data_set...')
dataloader = DataLoader(config.max_p_num, config.max_p_len, config.max_q_len, config.max_ch_len,
                        config.train_files, config.dev_files)

print('Converting text into ids...')
dataloader.convert_to_ids(vocab)

print('Initialize the model...')
model = Model(vocab, config)

print('Training the model...')
model.train(dataloader, config.epochs, config.batch_size, save_dir=config.model_dir, save_prefix=config.algo,
            dropout=config.dropout,evaluate=True)

print('====== Done with model training! ======')

Load vocab...
Load data_set...
Converting text into ids...
Initialize the model...
Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Total number of trainable parameters: 118253
Training the model...
Evaluating the model after epoch 1
{'testlen': 1155, 'reflen': 8033, 'guess': [1155, 1059, 966, 875], 'correct': [566, 340, 245, 195]}
ratio: 0.14378189966386856
Dev eval loss 14.753951867421469
Dev eval result: {'Bleu-1': 0.001270633225860841, 'Bleu-2': 0.0010284767124721227, 'Bleu-3': 0.0008860424878862743, 'Bleu-4': 0.0007962399991996641, 'Rouge-L': 0.09984377425203499}
Evaluating the model after epoch 2
{'testlen': 12247, 'reflen': 9534, 'guess': [12247, 12151, 12056, 11965], 'correct': [4428, 2687, 2072, 1785]}
ratio: 1.2845605202432049
Dev eval loss 13.75674819946289
Dev eval result

### Predict

We can use the test data for prediction

In [16]:
print('Load data_set and vocab...')
with open(os.path.join(config.vocab_dir, 'vocab.data'), 'rb') as fin:
        vocab = pickle.load(fin)

assert len(config.test_files) > 0, 'No test files are provided.'
dataloader = DataLoader(config.max_p_num, config.max_p_len, config.max_q_len, config.max_ch_len,test_files=config.test_files)

print('Converting text into ids...')
dataloader.convert_to_ids(vocab)
print('Restoring the model...')
model = Model(vocab, config)
model.restore(config.model_dir, config.algo)
print('Predicting answers for test set...')
test_batches = dataloader.next_batch('test', config.batch_size, vocab.get_word_id(vocab.pad_token), vocab.get_char_id(vocab.pad_token), shuffle=False)

model.evaluate(test_batches,result_dir=config.result_dir, result_prefix='test.predicted')

Load data_set and vocab...
Converting text into ids...
Restoring the model...
Predicting answers for test set...


Finally, we can check the file test.predicted to gain our answers we predicted!

In [ ]:
# {"question": "宋庆龄第二任丈夫是谁", "question_type": "ENTITY", "answers": ["刚看到解禁的文件,宋在上世纪三四十年代已经改嫁了,其丈夫是宋庆龄的秘书,这是一段婚姻已经被公布,宋曾向组织申请结婚,但是组织考虑其政治影响不同意结婚,但是可以同居,并在组织内公开以夫妻身份生活,这个相关文件能查到,这段婚姻是可查的。另外还有几段恋情盛传的,其中和其警卫员的恋情大众的认可比较高。"]}
# {"question": "勇者约吗漫画吹妖漫画", "question_type": "YES_NO", "answers": ["吹妖漫画网致力于挖掘精品原创漫画,专注于漫画业务,发现更多原创作者发掘优质原创漫画,为喜爱漫画的读者提供优质服务。利用互联网优越的大平台不断提升原创漫画的发展空间,让用户沉浸于漫画的喜悦当中。吹妖漫画app特色1、十万本热门漫画免费看,5000万动漫爱好者的共同选择。2、超高人气小说改编的漫画让你尽享视觉盛宴;3、众多一流原创漫画家精心上万本作品不断涌现、定期更新满足你的所有需求!4、无论你是小清新,还是重口味,都能找到合味口的开胃菜。5、这里不仅是寻找小伙伴的圣地,更是宅基腐的天堂。6、恐怖猎奇、热血搞笑应有尽有,御姐萝莉、幻想女仆、基腐联盟一网打尽,海量漫画免费看。"]}
# {"question": "wan口和lan口的区别", "question_type": "DESCRIPTION", "answers": ["在使用路由器上网时,我们会发现在路由器上,标注得有WAN口(有的路由器是Internet口)和LAN口(有的路由器标注的是1、2、3、4)。很多用户一看就晕了,根本就不知道WAN口与LAN口的区别,自然不知道应该怎么连接了。"]}
# {"question": "桂纶镁为什么不出现了", "question_type": "DESCRIPTION", "answers": ["在桂纶镁上大二时课余正好跟同学到西门町逛街买东西。当时,“蓝色大门”的剧组正好在街上寻找新演员第一部作品。"]}
# {"question": "美国航空怎么样", "question_type": "DESCRIPTION", "answers": ["美国航空编辑锁定"]}
# {"question": "qq微信密码", "question_type": "DESCRIPTION", "answers": ["不一样。两者是独立的密码,微信密码忘了的话可以通过手机短信验证码登陆。或者通过找回密码重新设置密码。"]}
# {"question": "机动车违反禁止标线指示的扣几分", "question_type": "ENTITY", "answers": ["违反禁令标志、禁止标线指示的。　　2013年新处罚标准:罚款100元,扣3分。　　原处罚标准:罚款100元,扣2分。"]}
# {"question": "凯乐石是哪个国家的品牌", "question_type": "ENTITY", "answers": ["官网还是英文的,好恶心,崇洋媚外的家伙。其实是国产,户外入门级厂家,价格偏高,质量还可以。"]}
# {"question": "痛风能吃什么", "question_type": "ENTITY", "answers": ["痛风病人可以吃猕猴桃,杨桃,樱桃,桃子,葡萄,西瓜,苹果,芒果,香蕉,哈密瓜,椰子,菠萝,山楂,桑葚,石榴,桂圆,橙子,梨,杏,柿子,柚子,橘子,柠檬,杨梅,榴莲,火龙果,枇杷,无花果。对于吃水果的禁忌,是没有什么决对的就不能吃什么。因为绝大多数水果里面的嘌呤含量都很少的。但是吃水果也不要过多。每天吃水果应该在150~200克为宜,最好饭前半小时吃水果。"]}
# {"question": "邪能解离器在哪", "question_type": "ENTITY", "answers": ["在苏拉玛左下角的燃烧军团大绿坑里,让我找一个解离器,然而任务提示的区域里什么都没有,打小怪也不掉。。。"]}
# {"question": "薛之谦偷吻郭雪芙哪期", "question_type": "ENTITY", "answers": ["薛之谦和郭雪芙是什么关系?薛之谦(JokerXue),1983年7月17日出生于上海,华语流行乐男歌手、影视演员、音乐制作人,毕业于格里昂酒店管理学院。"]}
# {"question": "下列不能缓解和消除肌肉酸痛的方法是", "question_type": "DESCRIPTION", "answers": ["A、擦伤B、肌肉拉伤C、开放性损伤D、闭和性损伤"]}
# {"question": "被搜索账号状态异常无法显示", "question_type": "DESCRIPTION", "answers": ["账号被举报次数过多存在异常"]}
# {"question": "柠檬泡水能天天喝吗", "question_type": "YES_NO", "answers": ["帧喝了快大半年了。脸白的很明显啊。而且豆印淡了。柠檬用白糖腌。天天泡着喝?柠檬水感光的。。。喝完了"]}
# {"question": "虎牙粉丝徽章怎么删除", "question_type": "DESCRIPTION", "answers": ["首先,你必须是12级的游戏认证主播,然后请进入虎牙直播个人中心—主播设置页面,提交昵称审核即可,官方会在三个工作日之内审核完毕。"]}
# {"question": "贪玩蓝月和蓝月传奇一样吗", "question_type": "YES_NO", "answers": ["贪玩一天开那么多区,玩个几天就合区。"]}
# {"question": "哪个保险公司好", "question_type": "ENTITY", "answers": ["哪个公司好第一要看的就是服务,其次是公司的财力,三是产品是否符合你的需求,四是代理人的素质。"]}
# {"question": "做菜网站哪个最好", "question_type": "ENTITY", "answers": ["美食天下,我也是不会做饭就看这个网站的,每道菜都是上面图下面有步骤的,很详细的介绍第一步。"]}
# {"question": "净水器市场怎么样", "question_type": "DESCRIPTION", "answers": ["反渗透净水器用水量大是其重要弊端,但如今市场上各品牌对净水器的废水比有所提升。目前,最为常见的是1:1的低废水比,即过滤出一杯纯水需要消耗一杯废水。另外,也有一些企业推出了纯水和废水比例为2:1和3:1的净水器。"]}
# {"question": "阳江有什么好玩的", "question_type": "ENTITY", "answers": ["2016-12-2114:351312/6"]}
# {"question": "泰拉瑞亚海草怎么用", "question_type": "DESCRIPTION", "answers": ["泰拉瑞亚中海草就是召唤海龟宠物的道具,钓鱼获得。电脑版:当海草在背包时,提出并单击即可召唤(死亡或重进存档将消失,需再次召唤),或者右键放入宠物栏(持续存在)。手机版:放入物品栏,单击召唤。"]}
# {"question": "婴儿身份证", "question_type": "DESCRIPTION", "answers": ["少儿身份证编辑锁定"]}
# {"question": "怎么出道做明星", "question_type": "DESCRIPTION", "answers": ["长的好看就行"]}
# {"question": "声音管理器在哪", "question_type": "ENTITY", "answers": ["开始控制面板"]}
# {"question": "汉仪字体有版权吗", "question_type": "YES_NO", "answers": ["学习研究之用没有版权,但是用途商业就会涉及版权,别人不告你就作罢,如果起诉你那是要赔钱的。国内著名的字体版权案:方正字体公司起诉暴雪游戏侵犯字体版权遭索赔4亿。"]}
# {"question": "共产党宣言的意义", "question_type": "DESCRIPTION", "answers": ["4.<<共产党宣言>>的历史意义:<<共产党宣言>>的发表,标志着科学社会主义(马克思主义)的诞生,无产阶级有了自己强大的思想武器,国际共产主义运动进入了新阶段.社会主义运动得到了蓬勃发展,是国际无产阶级的第一个战斗纲领,它对一代又一代立志为社会主义事业奋斗的人产生了极为深远的影响。"]}
# {"question": "念佛如何得到一心不乱", "question_type": "ENTITY", "answers": ["南怀谨:念佛如何得到一心不乱"]}
# {"question": "龙珠激斗账号", "question_type": "DESCRIPTION", "answers": ["共1页,到第页确定"]}
# {"question": "2017还怎么下载岛国片", "question_type": "DESCRIPTION", "answers": ["月风轻,风沙中渐次隐去的背影,已是匆忙过客。"]}
# {"question": "原件日语", "question_type": "DESCRIPTION", "answers": ["在沪江关注日语的沪友zhcui遇到了一个关于日语翻译的疑惑,已有4人提出了自己的看法。"]}
# {"question": "如何擦纱窗", "question_type": "DESCRIPTION", "answers": ["3、用面糊除纱窗上的油污"]}
# {"question": "心脏支架手术后寿命", "question_type": "ENTITY", "answers": ["专家观点:关于心脏支架术后患者寿命的问题,如果能严格按照医生的要求做好药物和生活的护理,养成良好的生活习惯,合理膳食,并注意情绪的控制,病情是可以得到有效控制的,术后的生存时间不会受到很大影响,和正常人一样活到预期的寿命。"]}
# {"question": "经纪代理服务增值税", "question_type": "DESCRIPTION", "answers": ["2016年3月23日,财政部、国家税务总局正式颁布《关于全面推开营业税改征增值税试点的通知》(财税〔2016〕36号),自2016年5月1日起,全面推开营改增,将试点范围扩大到建筑业、房地产业、金融业、生活服务业。"]}
# {"question": "我的世界mac版多少钱", "question_type": "ENTITY", "answers": ["当前位置:首页→Mac游戏→模拟游戏→我的世界Mac版启动器V3.5.1"]}
# {"question": "如何把安卓手机资料导入iphone", "question_type": "DESCRIPTION", "answers": ["可以使用百度网盘。先在安卓手机上安装百度网盘,然后登陆,把照片备份到网盘时面。然后在IPHONE上安装百度网盘,登陆帐号,可以随时把照片下载下来。"]}
# {"question": "什么是小规模纳税人和一般纳税人的区别", "question_type": "DESCRIPTION", "answers": ["①使用发票不同。小规模纳税人销售只能使用普通发票,不能使用增值税专用发票,购买货物与一般纳税人相同,可以收普通发票也能收增值税专用发票,二者收取增值税专用发票后帐务处理不同.一般纳税人按价款部分入成本,税款部分入\"应交税金--应交增值税--进项税额\"帐户;小规模纳税人则按全额进入成本.②应交税金的计算方法不同。一般纳税人按\"抵扣制\"计算税金,即按销项减进项后的余额交税.小规模纳税人按销售收入除于(1+适用税率)后的金额再乘税率计算应交税金,工业6%,商业4%。③税率不同,一般纳税人分为0税率、13%税率、17%税率。小规模纳税人,商业企业按4%;工业企业按6%,(免税的除外)。1、年应税销售额在100万元以上2、会计核算健全,年应税销售额不足100万元、但超过30万元。"]}
# {"question": "12306账号注销", "question_type": "DESCRIPTION", "answers": ["(1)注册12306账号的用户可以将账号注销,但是,网上和客服是无法办理注销的,用户只能拿着自己的二代居民身份证到火车站售票窗口办理注销手续。(2)有没有其他简单省时的办法?办理注销手续,一是觉得以后可能用不到了,二是担心自己的身份信息可能泄露出去,因为,注册时填写的都是真实信息,泄露出去可能会对用户的生活造成一定的影响。那么,可不可以将注册的信息修改一下呢?这样不是也同样起到了注销的效果了。(3)如何修改信息?用户登录自己的账号后,点击查看个人信息,然后选择编辑,将姓名和身份证号进行修改后点击保存即可。"]}
# {"question": "为什么要重写hashcode和equals", "question_type": "DESCRIPTION", "answers": ["1、重写equals方法时需要重写hashCode方法,主要是针对Map、Set等集合类型的使用;a:Map、Set等集合类型存放的对象必须是唯一的;b:集合类判断两个对象是否相等,是先判断equals是否相等,如果equals返回TRUE,还要再判断HashCode返回值是否ture,只有两者都返回ture,才认为该两个对象是相等的。2、由于Object的hashCode返回的是对象的hash值,所以即使equals返回TRUE,集合也可能判定两个对象不等,所以必须重写hashCode方法,以保证当equals返回TRUE时,hashCode也返回Ture,这样才能使得集合中存放的对象唯一。"]}
# {"question": "直播h1z1需要什么配置", "question_type": "DESCRIPTION", "answers": ["4300块如果非得4000块以下,可以把显卡换成gtx1050950元"]}
# {"question": "饮用葡萄酒的室温通常指的是什么?", "question_type": "DESCRIPTION", "answers": ["应该是葡萄酒建议饮用温度吧。干红的话一般在16到22度左右"]}
# {"question": "dnf测试伤害在哪", "question_type": "DESCRIPTION", "answers": ["按M看伤害分析"]}
# {"question": "集体户口怎么领结婚证", "question_type": "DESCRIPTION", "answers": ["准备证件:双方身份证原件、集体户口簿内的户口卡或者加盖单位印章的记载其户籍情况的户口簿复印件。"]}
# {"question": "eln文件怎么打开", "question_type": "DESCRIPTION", "answers": ["题目好像没写正确。"]}
# {"question": "8zrfxe发动机怎么样", "question_type": "DESCRIPTION", "answers": ["双擎的发动机指标和普通发动机不同,车辆行驶时还要加上电动机的功率,看这个没有意义【汽车有问题,问汽车大师。4S店专业技师,10分钟解决。"]}
# {"question": "基因重组发生在什么时期", "question_type": "ENTITY", "answers": ["减数第一次分裂的四分体时期和减数第一次分裂后期"]}
# {"question": "wps表格中的文字怎么上下居中", "question_type": "DESCRIPTION", "answers": ["选中需要居中的文字,单击鼠标右键,在出现的快捷菜单中选择“单元格对齐方式”再选择居中即可。选择需要居中的文字,单击鼠标右键,找到表格属性,里面有左对齐、居中、右对齐,选择居中即可。"]}
# {"question": "s990和s925的区别", "question_type": "DESCRIPTION", "answers": ["摘要s990和s925哪个好?990和925都代表银的纯度,s990比s925高一点。一般情况下,银饰品都采用925,因为里面添加了别的金属元素,银饰可以被打造成各种形状。"]}
# {"question": "wow时光徽章实时价格", "question_type": "ENTITY", "answers": ["1.未来还会继续增添监控:欧、台、韩和国服的时光徽章价格走势。2.地区与地区间价格对比功能。"]}
# {"question": "窦性心律过缓的原因", "question_type": "DESCRIPTION", "answers": ["窦性心律过缓指如果心电图是窦性,低于60次/分为心律过缓,而正常的窦性心律是60-100次/分。窦性心律过缓本身一般不需处理,主要是针对病因进行治疗。"]}
# {"question": "东方爱婴收费价格多少", "question_type": "ENTITY", "answers": ["暂无会员评价"]}
# {"question": "比特币市场价格", "question_type": "ENTITY", "answers": ["比特币中国发布停止交易公告后,命运已定。"]}
# {"question": "圆通快递保价费怎么算", "question_type": "DESCRIPTION", "answers": ["100元保价2元钱,比如说你的货值500元你可交10元保价最好上限好像是2000元"]}
# {"question": "古力娜扎是穆斯林吗", "question_type": "YES_NO", "answers": ["首先得信教,然后才能谈教法好吗?维族不全是穆斯林,正像汉族也有穆斯林一样。"]}
# {"question": "给小组起个好听名字", "question_type": "ENTITY", "answers": ["天翊(有翱翔天际,大展宏图之意。)凯瑞(有气质高贵,潇洒大气之意。)"]}
# {"question": "我国外贸出口如何应对国际经贸环境的变化", "question_type": "DESCRIPTION", "answers": ["2010年国际贸易出现恢复性快速反弹;(5)国际大宗商品价格稳中有降,市场需求尚未强势;2013年,受世界经济低迷的影响,国际大宗商品价;从国内看,中国经济开局平稳,但仍存下行压力;经济全球化的今天,任何国家都势必会受到国际经济贸;(1)资源环境约束日益强化;我国对外贸易发展仍主要依靠资源、能源、土地和环境;-10-;石、铝土矿、铜矿等重要能源资源消费对进口"]}
# {"question": "上海石油化工交易中心申请模拟账号", "question_type": "DESCRIPTION", "answers": ["上海石油化工交易中心***相关产品"]}
# {"question": "倩女幽魂手游表情代码", "question_type": "ENTITY", "answers": ["倩女幽魂手游隐藏表情代码_倩女幽魂手游_琵琶网"]}
# {"question": "为什么没有北瓜", "question_type": "DESCRIPTION", "answers": ["没有"]}
# {"question": "tgp王者荣耀在哪", "question_type": "ENTITY", "answers": ["王者荣耀没有了"]}
# {"question": "两参人员是什么意思", "question_type": "DESCRIPTION", "answers": ["参加过14次以上局部战斗,或参加过核试验人员,这里“两参”是重点优抚关爱对象。"]}
# {"question": "狗狗安乐死多少钱", "question_type": "ENTITY", "answers": ["200——500不等,越大越贵"]}
# {"question": "萌芽投稿成功几率", "question_type": "ENTITY", "answers": ["《萌芽》,青春文学标杆,几代作家从这里起步。2016年全面升级,继续陪伴你。给《萌芽》投稿吧!我们承诺来稿必看,用稿必酬,发表后寄样刊。投稿请留联系方式(名字/电话/地址/邮箱),建议初次投稿者至少要看完一本近期《萌芽》,注意契合度。"]}
# {"question": "闲鱼交易卖家会被骗吗", "question_type": "YES_NO", "answers": ["没有任何说法保证闲鱼平台卖家就不会被骗。"]}
# {"question": "徐州有哪些大学", "question_type": "ENTITY", "answers": ["坐落在徐州的大学(本科院校)有:中国矿业大学(徐州),全国排名60左右江苏师范大学,全国排名200左右徐州医科大学,全国排名400左右徐州工程学院,全国排名500左右军校和专科高职院校不在此范围之内。"]}
# {"question": "结婚19年是什么婚", "question_type": "ENTITY", "answers": ["19年是印花婚"]}
# {"question": "夏威夷果的功效与作用", "question_type": "DESCRIPTION", "answers": ["夏威夷果的功效与作用"]}
# {"question": "a4尺寸大小", "question_type": "ENTITY", "answers": ["成品尺寸不一样"]}
# {"question": "古琴有几根弦", "question_type": "ENTITY", "answers": ["现代的古琴为七弦"]}
# {"question": "b站弹幕上限的标准从哪里查询", "question_type": "DESCRIPTION", "answers": ["点击播放器右边的设置按钮之后在设置底端调整即可"]}
# {"question": "工伤报销比例", "question_type": "ENTITY", "answers": ["工伤保险缴费比例已经确定,职工本人不需要缴费,单位缴费比例根据行业不同而确定。"]}
# {"question": "antipodes跟伊索哪个好", "question_type": "ENTITY", "answers": ["5、Aesop/伊索樱草洁净敷面膜/樱草面膜"]}
# {"question": "丹霞山为什么那么像阴茎", "question_type": "DESCRIPTION", "answers": ["过期自动退"]}
# {"question": "华为重启后的解锁密码", "question_type": "DESCRIPTION", "answers": ["华为手机开机密码忘了可以通过对手机双清的方法来清除解锁密码:手机彻底的关机。彻底关机之后,同时按住音量增加键+电源键一起按,不要松,过一会儿进入recovery模式(recovery模式下:音量键为光标选择键,可以用来移动光标,电源键则是确认键)。进入recovery模式之后进行双清,找到【wipedata/factoryreset】和【wipecachepartition】这两项就可以,选择【wipedata/factoryreset】,然后按开机键确认,再选择【yes-deletealldata】(是的-清空所有数据)即可。然后再选择【rebootsystemnow】,按开机键确认后启动手机就不需要解锁密码了。"]}
# {"question": "手机wifi连不上", "question_type": "DESCRIPTION", "answers": ["1以上就是小编对手机连不上wifi是怎么回事的解答,相信大家对此已经有所了解了。只要根据你自己遇到的实际情况找到问题所在,所有的问题就可以迎刃而解了。"]}
# {"question": "g-shock怎么调时间", "question_type": "DESCRIPTION", "answers": ["按住左上角的按钮大概5秒钟以上会出现Set闪烁再按左下角的按钮切换时间及日期的设置"]}
# {"question": "如何装打印机驱动", "question_type": "DESCRIPTION", "answers": ["安装打印机方法:1开始--控制面板--添加打印机。2点击下一步,(如果打印机在本机,选择此计算机连本地打印机,之后点下一步3、选择打印机插孔,一般的usb和ltp1,插口,正常你的这个是ltp插口的。,选择完毕点下一步)。4、点下一步,在出现一个对话框,在厂商那里找惠普,在型号那里找到对应型号。5、点击下一步,下一步,完成。6、插上打印机,通电,自动检测完成安装。"]}
# {"question": "血尿素氮偏高的原因", "question_type": "DESCRIPTION", "answers": ["较易受饮食、肾血流量的影响,如有蛋白质分解因素—感染,肠道出血、甲亢等可使尿素氮升高。肾小球滤过率下降至正常的1/2~~1/3时,尿素氮逐步升高,一般情况下血尿素氮与血肌酐的比值是10.1,比值升高的原因有胃肠道出血,溶血,心功能不全和组织分解增强(烧伤、高热、肾上腺皮质激素治疗等),多为肾前因素引起,比值降低见于蛋白质摄入过少,严重肝肾功能不全等。"]}
# {"question": "盲僧如何光速摸眼", "question_type": "DESCRIPTION", "answers": ["首先所有技能都设置智能施法,然后把眼放在你最顺手的那个数字键相对应的格子里,先练好快速摸眼,摸眼的时候鼠标别动,鼠标悬停你要位移的地方,插眼再W,别同时按插眼和W,有先后顺序的,手速快点。摸眼快了以后回旋踢就不是什么问题了,关键得理智。"]}
# {"question": "3dmax2012中文版序列号", "question_type": "DESCRIPTION", "answers": ["3dmax安装说明序列号激活码_表格/模板_实用文档。3dmax激活码"]}
# {"question": "婚后共同财产一方去世", "question_type": "DESCRIPTION", "answers": ["由此可见,夫妻一方死亡,并不能以所有的夫妻财产为其遗产,而应当先认定夫妻共同财产,从中划出一半为生存方所有,另一半夫妻共同财产,才可认定为死亡方的个人财产。"]}
# {"question": "什么是springmvc", "question_type": "DESCRIPTION", "answers": ["只要是新的项目,要用到MVC的,基本上都改用springmvc了。"]}
# {"question": "999小儿止咳糖浆怎么办服用", "question_type": "DESCRIPTION", "answers": ["以上分享了治疗宝宝咳嗽厉害小妙招,家长可以多掌握一些治疗宝宝咳嗽的按摩方法和食疗方法,这样在宝宝轻度咳嗽的时候,就可以用这些方法调理,如果宝宝咳嗽厉害了,一定要用药物治疗,家中常备999小儿止咳糖浆,用药治疗的同时辅以食疗和按摩方法,相信宝宝咳嗽很快就会好的。"]}
# {"question": "小五离开天天向上是哪一期", "question_type": "ENTITY", "answers": ["12月18日晚天天向上录制是小五最后一期,现场小五跪地行大礼进行三分钟的告别仪式。"]}
# {"question": "龙珠顺序", "question_type": "ENTITY", "answers": ["龙珠,龙珠z,龙珠gt"]}
# {"question": "安化黑茶价格多少一斤", "question_type": "ENTITY", "answers": ["近日铑黑多少钱一斤"]}
# {"question": "法术伤害结果好还是法术伤害好", "question_type": "DESCRIPTION", "answers": ["每个法术技能是由法术伤害然后通过公式计算造成的伤害的。"]}
# {"question": "gta5热咖啡怎么按键", "question_type": "DESCRIPTION", "answers": ["游戏相关知识游戏"]}
# {"question": "花呗怎么套出来", "question_type": "DESCRIPTION", "answers": ["百度搜索祥雨福"]}
# {"question": "如何清除远程桌面连接记录", "question_type": "DESCRIPTION", "answers": ["清除远程桌面访问痕迹。"]}
# {"question": "加百利史诗碎片怎么兑换", "question_type": "DESCRIPTION", "answers": ["碎片不够换"]}
# {"question": "输入什么命令可以查本机号码", "question_type": "DESCRIPTION", "answers": ["按0再按拨号键"]}
# {"question": "微信怎么撤回消息", "question_type": "DESCRIPTION", "answers": ["微信只能撤回2分钟内发出的信息,超出2分钟的信息不能撤回,如图。微信撤回发出信息方法:1)打开微信,在跟朋友聊天的窗口,长按已经发送出去的最后消息(2分钟内)。2)出现选项,选择【撤回】,这样就可以撤回一条在2分钟内发送的最后一条消息了。"]}
# {"question": "西安三星空调售后电话", "question_type": "ENTITY", "answers": ["欢迎访问~{西安三星冰箱全国售后维修服务各点}—咨询电话"]}
# {"question": "成都人流哪里做好", "question_type": "ENTITY", "answers": ["去正规医院"]}
# {"question": "沈阳建筑大学是211吗", "question_type": "YES_NO", "answers": ["沈阳建筑大学不是211,也不是985。沈阳建筑大学是以建筑、土木、机械等学科为特色,以工为主,工、管、理、文、农、法、艺术等学科门类协调发展的省部共建高等学校,原隶属于国家建设部,2000年在国家办学管理体制调整中划归辽宁省管理。2010年,辽宁省政府与国家住房和城乡建设部在北京签署了共建沈阳建筑大学协议,为学校创建有特色、高水平建筑大学奠定了坚实的基础,为进一步密切学校与行业及行业主管部门的联系搭建了新的平台。"]}
# {"question": "凸窗和飘窗的区别", "question_type": "DESCRIPTION", "answers": ["一份图纸里门窗表里给的是凸窗,这个怎么画。我用飘窗画出来窗子的两边都是墙而不是玻璃窗。"]}
# {"question": "如何制定目标", "question_type": "DESCRIPTION", "answers": ["扫描二维码下载"]}
# {"question": "扑朔迷离的意思", "question_type": "DESCRIPTION", "answers": ["成语大全查询结果由aies.cn提供[复制结果]"]}
# {"question": "怡宝出问题了吗", "question_type": "YES_NO", "answers": ["没事儿的,把心放肚子里吧!不会对你的身体健康带来任何危害。"]}
# {"question": "明成祖是谁", "question_type": "ENTITY", "answers": ["因为所以科学道理"]}